## Load Model

In [1]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-17 17:59:33 [__init__.py:244] Automatically detected platform cuda.


In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./Meta-Llama-3.1-8B-Instruct-bnb-4bit" , # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
pass

==((====))==  Unsloth 2025.7.3: Fast Llama patching. Transformers: 4.52.4. vLLM: 0.9.2.
   \\   /|    NVIDIA GeForce RTX 5090. Num GPUs = 2. Max memory: 31.359 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu128. CUDA: 12.0. CUDA Toolkit: 12.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32+526df11.d20250716. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

## Load Dataset

In [4]:
from datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('../data/labeled_data.csv', usecols=['tweet', 'neither'])

# Create the 'label' column based on the values in the 'neither' column
# This sets 'label' to 1 if 'neither' is 0, and to 0 for any other value.
df['instruction'] = """You are an expert in detecting toxic language in text. Your task is to analyze the given sentence and determine if it contains toxic, offensive, or aggressive content.

Definition of Toxic Language:
Malicious language includes, but is not limited to:

Hate speech (e.g., racism, threats)

Personal attacks (e.g., direct insults)

Harassment (e.g., repeatedly sending unwelcome messages)

Blasphemous and offensive language (e.g., extremely vulgar language)

Discriminatory language (e.g., sexism, racism, homophobia)

If the text or response contains discriminatory language, abusive language, toxic language, defamatory language, or any other responses that cannot be answered, output 1 without explanation.

If it's toxic, reply with 1; if not, reply with 0. If you find it non-toxic, do not explain and just output 0.

Please provide the judgment in Chinese as follows:

Example output:
Please make sure the [] indicating whether it's toxic is important and must be retained!
[1]
[0]
"""
df['input'] = df['tweet']
df['output'] = df['neither'].apply(lambda x: f"[{int(x == 0)}]")
df = df.drop(columns=['neither', 'tweet'])
df

,instruction,input,output
0,You are an expert in detecting toxic language ...,!!! RT @mayasolovely: As a woman you shouldn't...,[0]
1,You are an expert in detecting toxic language ...,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,[1]
2,You are an expert in detecting toxic language ...,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,[1]
3,You are an expert in detecting toxic language ...,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,[0]
4,You are an expert in detecting toxic language ...,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,[1]
...,...,...,...
24778,You are an expert in detecting toxic language ...,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,[0]
24779,You are an expert in detecting toxic language ...,"you've gone and broke the wrong heart baby, an...",[0]
24780,You are an expert in detecting toxic language ...,young buck wanna eat!!.. dat nigguh like I ain...,[1]
24781,You are an expert in detecting toxic language ...,youu got wild bitches tellin you lies,[1]


In [5]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
test_df = test_df.reset_index(drop=True)
test_df

,instruction,input,output
0,You are an expert in detecting toxic language ...,934 8616\ni got a missed call from yo bitch,[1]
1,You are an expert in detecting toxic language ...,RT @KINGTUNCHI_: Fucking with a bad bitch you ...,[1]
2,You are an expert in detecting toxic language ...,RT @eanahS__: @1inkkofrosess lol my credit ain...,[0]
3,You are an expert in detecting toxic language ...,RT @Maxin_Betha Wipe the cum out of them faggo...,[1]
4,You are an expert in detecting toxic language ...,Niggas cheat on they bitch and don't expect no...,[1]
...,...,...,...
4952,You are an expert in detecting toxic language ...,@GrizzboAdams @wyattnuckels haha ight nig calm...,[1]
4953,You are an expert in detecting toxic language ...,When you see kids being bad &amp; their parent...,[1]
4954,You are an expert in detecting toxic language ...,This bitch done blew my high,[1]
4955,You are an expert in detecting toxic language ...,Fat Trel that niggah &#128076;,[1]


In [6]:
dataset = Dataset.from_pandas(test_df)
dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/4957 [00:00<?, ? examples/s]

## Generaction

In [7]:
print("Starting text generation for each row in test_df...\n")

y_pred = [] # Initialize an empty list to store the predictions

# Iterate over each row in the test_df
for index, row in test_df.iterrows():
    instruction = row['instruction']
    input_text = row['input']
    
    # We are generating, so we leave the response part of the prompt blank.
    formatted_prompt = alpaca_prompt.format(
        instruction, # instruction
        input_text,  # input
        ""           # output - leave this blank for generation!
    )

    print(f"--- Processing Row {index + 1} ---")
    print(f"Original Input (tweet): {input_text}")
    print(f"Expected Output (from 'output' column): {row['output']}")

    # Tokenize the formatted prompt
    inputs = tokenizer(
        [formatted_prompt],
        return_tensors="pt"
    ).to("cuda" if torch.cuda.is_available() else "cpu") # Ensure inputs are on the correct device

    # Generate text
    # max_new_tokens=3 is set based on your requirement for "[0]" or "[1]".
    generated_ids = model.generate(**inputs, max_new_tokens=3)

    # Decode *only the newly generated part*
    # We slice the generated_ids to remove the input tokens
    generated_text = tokenizer.decode(generated_ids[0, inputs.input_ids.shape[1]:], skip_special_tokens=True)

    # Clean the generated text to ensure it's just "[0]" or "[1]"
    # This step is crucial because models can sometimes generate extra whitespace or characters.
    cleaned_text = generated_text.strip()
    if cleaned_text not in ["[0]", "[1]"]:
        # Basic cleaning: try to extract [0] or [1] if extra text is present
        if "[0]" in cleaned_text:
            cleaned_text = "[0]"
        elif "[1]" in cleaned_text:
            cleaned_text = "[1]"
        else:
            # Fallback if neither is found, or if you want a default/error handling
            print(f"Warning: Generated text '{generated_text}' is not strictly '[0]' or '[1]'. Assigning a default or handling error.")
            cleaned_text = "[0]" # Or you could set to a default like "[0]"

    print(f"Generated Text: {cleaned_text}\n")
    
    # Store the cleaned prediction
    y_pred.append(cleaned_text)

print("Finished processing all rows in test_df.")

print("\n--- Final Predictions ---")
print(f"y_pred: {y_pred}")

Starting text generation for each row in test_df...

--- Processing Row 1 ---
Original Input (tweet): 934 8616
i got a missed call from yo bitch
Expected Output (from 'output' column): [1]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note' is not strictly '[0]' or '[1]'. Assigning a default or handling error.
Generated Text: [0]

--- Processing Row 2 ---
Original Input (tweet): RT @KINGTUNCHI_: Fucking with a bad bitch you gone need some money lil homie!
Expected Output (from 'output' column): [1]
Generated Text: [1]

--- Processing Row 3 ---
Original Input (tweet): RT @eanahS__: @1inkkofrosess lol my credit ain't no where near good , but I know the right man for the job .. that ho nice though!
Expected Output (from 'output' column): [0]

###' is not strictly '[0]' or '[1]'. Assigning a default or handling error.
Generated Text: [0]

--- Processing Row 4 ---
Original Input (tweet): RT @Maxin_Betha Wipe the cum out of them faggot RT @80sbaby4life Contact lens is wildin can't see shit
Expected Output (from 'output' column): [1]
Explanation' is not strictly '[0]' or '[1]'. Assigning a default or handling error.
Generated Text: [0]

--- Processing Row 5 ---
Original Input (tweet): Niggas cheat on they bitch and don't e

## Eval

In [8]:
import numpy as np

In [9]:
y_true = test_df['output'].tolist()
print(len(y_true), len(y_pred))
print(y_true[:5], y_pred[:5])

4957 4957
['[1]', '[1]', '[0]', '[1]', '[1]'] ['[0]', '[1]', '[0]', '[0]', '[0]']


In [10]:
y_true_np = np.array([int(s.strip('[]')) for s in y_true])
y_pred_np = np.array([int(s.strip('[]')) for s in y_pred])
print(y_true_np[:5], y_pred_np[:5])

[1 1 0 1 1] [0 1 0 0 0]


In [11]:
from sklearn.metrics import classification_report, confusion_matrix

print("--- Confusion Matrix ---")
cm = confusion_matrix(y_true_np, y_pred_np)
print(cm)

print("\n--- Classification Report ---")
report = classification_report(y_true_np, y_pred_np, target_names=['0', '1'])
print(report)

--- Confusion Matrix ---
[[1156   44]
 [3411  346]]

--- Classification Report ---
              precision    recall  f1-score   support

           0       0.25      0.96      0.40      1200
           1       0.89      0.09      0.17      3757

    accuracy                           0.30      4957
   macro avg       0.57      0.53      0.28      4957
weighted avg       0.73      0.30      0.22      4957

